## Country comparison

In [1]:
import altair as alt
from altair import datum
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import HTML, Markdown as md

from src.data.read_dataset import get_processed_dataset
from src.visualization import visualize as viz

ecdc = get_processed_dataset('ECDC')

In [2]:
today = ecdc.groupby('Country').first()
today = today[(today.popData2019 > 1e6) & (today.TotalCases > 1000)]

In [3]:
fields = ['TotalCasesPer1MPop', 'TotalDeathsPer1MPop', 'CasesPer1MPop', 'DeathsPer1MPop', 'TotalCases', 'TotalDeaths', 'Lethality', 'cases', 'deaths']

@interact(campo=fields, giorni_indietro=(-30,0))
def plot(campo, giorni_indietro=0):
    day = ecdc.groupby('Country').nth(-giorni_indietro).reset_index()
    day = day[(day.popData2019 > 1e6) & (day.TotalCases > 1000)]
    c = alt.Chart(day).mark_bar().encode(
        x=campo,
        y=alt.Y('Country:N', sort='-x'),
        #tooltip=['dateRep', 'Country', 'popData2019'] + fields
    ).transform_window(
        rank=f'rank({campo})',
        sort=[alt.SortField(campo, order='descending')]
    ).transform_filter(
        (alt.datum.rank < 21)
    ).properties(
        height=600,
        width=800
    )#.properties(title=day.dateRep.iloc[0]).interactive()
    return c

interactive(children=(Dropdown(description='campo', options=('TotalCasesPer1MPop', 'TotalDeathsPer1MPop', 'Cas…